In [ ]:
import pandas as pd
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import hashlib
from sklearn.model_selection import train_test_split

In [ ]:
def load_housing_data(housing_path):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [ ]:
housing = load_housing_data("datasets")
housing.head()

In [ ]:
housing.info()

In [ ]:
housing['ocean_proximity'].value_counts()

In [ ]:
housing.describe()

In [ ]:
housing.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
def split_train_test(data, test_ratio):
    np.random.seed(42)
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)
print(len(train_set), "train + ", len(test_set), "test")

In [ ]:
# hash approach to split T/T

def test_set_check(identifier, test_ratio, hash):
    return hash(np.int64(identifier)).digest()[-1] < (256 * test_ratio)

def split_train_test_by_id(data, test_ratio, id_column, hash=hashlib.md5):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio, hash))
    return data.loc[~in_test_set], data.loc[in_test_set]

housing_with_id = housing.reset_index() # add an 'index' column (row number)
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")
print(len(train_set), "train + ", len(test_set), "test")

housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")
print(len(train_set), "train + ", len(test_set), "test")

In [ ]:
# sklearn approach to split T/T

train_set, test_set = train_test_split(housing, test_size = 0.2, random_state = 42)
print(len(train_set), "train + ", len(test_set), "test")

In [ ]:
# stratified sampling (median income)

# create the strata
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)
housing["income_cat"].hist()

# Stratified Shuffle Split
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    
# check proportions
print(strat_train_set["income_cat"].value_counts() / len(strat_train_set))
print(strat_test_set["income_cat"].value_counts() / len(strat_test_set))

# remove income_cat
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis = 1, inplace=True)
strat_train_set.columns

In [ ]:
# create a copy of the train data
housing = strat_train_set.copy()

In [ ]:
# plot lat/long
housing.plot(kind = "scatter", x = "longitude", y = "latitude")
housing.plot(kind = "scatter", x = "longitude", y = "latitude", alpha = 0.1) # shows density of points better

In [ ]:
# more complex plot
housing.plot(kind = "scatter", x = "longitude", y = "latitude", alpha = 0.4,
            s=housing["population"] / 100, label = "population", figsize = (10, 7),
            c="median_house_value", cmap=plt.get_cmap("jet"), colorbar = True)
plt.legend()

In [ ]:
# get help on plot
housing.plot?

In [ ]:
# correlation matrix
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# pandas scatter matrix
from pandas.plotting import scatter_matrix

attributes = ['median_house_value', 'median_income', 'total_rooms', 'housing_median_age']
scatter_matrix(housing[attributes], figsize = (12, 8))

In [ ]:
# scatter plot
housing.plot(kind = "scatter", x = "median_income", y = "median_house_value", alpha = 0.1)

In [ ]:
# feature engineering

housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"] / housing["households"]

# get correlations again
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# recopy the original stratified sample to start fresh
housing = strat_train_set.drop("median_house_value", axis = 1)
housing_labels = strat_train_set["median_house_value"].copy()

# 3 options to handle the missing values in total_bedrooms
housing.dropna(subset = ["total_bedrooms"])
housing.drop("total_bedrooms", axis = 1)
median = housing["total_bedrooms"].median() # note will need to use this to impute on test too
housing["total_bedrooms"].fillna(median, inplace=True)

In [ ]:
# sklearn imputer class
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy = "median")
housing_num = housing.drop("ocean_proximity", axis = 1) # need to remove categorical features to run imputer
imputer.fit(housing_num)

print(imputer.statistics_) # note the values saved here
print(housing_num.median().values)

In [ ]:
# using the imputer
X = imputer.transform(housing_num) # output is a numpy array
housing_tr = pd.DataFrame(X, columns = housing_num.columns) # to go back to df

In [ ]:
# encoding categoricals with factorize
housing_cat = housing["ocean_proximity"]
housing_cat.head()

housing_cat_encoded, housing_categories = housing_cat.factorize()
print(housing_cat_encoded[:10])
housing_categories

In [ ]:
# encoding categoricals with OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1)) # fit_transform expects 2D array
housing_cat_1hot # note output is scipy sparse matrix

# can turn sparse matrix into array if needed
housing_cat_1hot.toarray()

In [ ]:
# NOTE: can go directly from text to onehot encoding using OneHotEncoder
from future_encoders import OneHotEncoder

cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat.values.reshape(-1, 1))
housing_cat_1hot # note output is a sparse matrix

In [ ]:
# create custom transformation class

from sklearn.base import BaseEstimator, TransformerMixin # TransformerMixin is used for fit_transform, 
# BaseEstimator is used for get_params() and set_params()

rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
# transformation pipelines - takes a list of name, estimator pairs
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('imputer', Imputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# We can get a more complete process by allowing the ingestion of a df, not a numpy matrix. Do so we write a custom
# transformer

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X[self.attribute_names].values

In [ ]:
# add the above to the pipeline and create cat pipeline

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', Imputer(strategy='median')),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler())
])

cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attribs)),
    ('cat_encoder', OneHotEncoder(sparse=False)),
])

In [ ]:
# FeatureUnion - to combine the two pipelines
from sklearn.pipeline import FeatureUnion

full_pipeline = FeatureUnion(transformer_list = [
    ("num_pipeline", num_pipeline),
    ("cat_pipeline", cat_pipeline),
])

housing_prepared = full_pipeline.fit_transform(housing)
print(housing_prepared)
print(housing_prepared.shape)

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Test out linear regression
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
# measure rmse
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_predictions, housing_labels)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
# DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_predictions, housing_labels)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

In [ ]:
# cross validation
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
tree_rmse_scores = np.sqrt(-scores) # output is array

In [ ]:
# function to display model performance
def display_scores(scores):
    print("Scores: ", scores)
    print("Mean: ", scores.mean())
    print("SD: ", scores.std())
    
display_scores(tree_rmse_scores)

In [ ]:
# evaluate linreg
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
display_scores(np.sqrt(-scores))

In [ ]:
# build RF model
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)
scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring = "neg_mean_squared_error", cv = 10)
display_scores(np.sqrt(-scores))

In [ ]:
# save your model - can use pickle module or joblib
from sklearn.externals import joblib

# joblib.dump(forest_reg, "rf.pkl")
# rf_loaded = joblib.load(rf.pkl)

In [ ]:
# gridsearch
from sklearn.model_selection import GridSearchCV

param_grid = [
    {"n_estimators": [3, 10, 30], "max_features": [2, 4, 6, 8]},
    {"bootstrap": [False], "n_estimators": [3, 10], "max_features": [2, 3, 4]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring="neg_mean_squared_error")
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
# assess best model
print(grid_search.best_params_)
print(grid_search.best_estimator_) # note if refit=True in GridSearchCV then best model is refit on full data
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
# Analyse results from RF model
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = cat_pipeline.named_steps["cat_encoder"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse = True)

In [ ]:
# evaluate on test
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis = 1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)

final_predictions = final_model.predict(X_test_prepared)

final_rmse = np.sqrt(mean_squared_error(final_predictions, y_test))
final_rmse

In [ ]:
# EXERCISES

In [ ]:
# SVM regressor (sklearn.svm.SVR): kernel = linear (various C), kernel = "rbf" (various C, gamma)
from sklearn.svm import SVR
param_grid = [
    {"kernel": ["linear"], "C": [1, 10, 100, 1000]},
    {"kernel": ["rbf"], "C": [1, 10, 100, 1000], "gamma": [0.001, 0.01, 0.1, 1.0]},
]

svm = SVR()
grid_search = GridSearchCV(svm, param_grid, cv=5, scoring="neg_mean_squared_error") #n_jobs?, verbose?
grid_search.fit(housing_prepared, housing_labels)

In [ ]:
print(np.sqrt(-grid_search.best_score_))
print(grid_search.best_params_)

In [ ]:
# Use RandomizedSearchCV
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

param_dist = {
    "kernel": ["linear", "rbf"],
    "C": reciprocal(20, 200000), 
    "gamma": expon(scale = 1.0),
}

svm = SVR()
rnd_search = RandomizedSearchCV(svm, param_dist, n_iter = 10, n_jobs = 4, verbose =2, random_state = 42, 
                                cv=3, scoring="neg_mean_squared_error") 
rnd_search.fit(housing_prepared, housing_labels)

In [ ]:
print(np.sqrt(-rnd_search.best_score_))
print(rnd_search.best_params_)

In [ ]:
# Add a transformer to select only the most important attributes

def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])

class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self
    def transform(self, X):
        return X[:, self.feature_indices_]

In [ ]:
print(np.argpartition(np.array(feature_importances), -5))
print(np.argpartition(np.array(feature_importances), -10))
print(np.sort(np.argpartition(np.array(feature_importances), -12)[-5:]))

In [ ]:
np.argpartition?

In [ ]:
# test transformer
k = 5
top_k_feature_indices = indices_of_top_k(feature_importances, k)
top_k_feature_indices
print(np.array(attributes)[top_k_feature_indices])
sorted(zip(feature_importances, attributes), reverse=True)[:k]

In [ ]:
# Create a single pipeline to do data preparation plus prediction

prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k)),
    ('svm_reg', SVR(**rnd_search.best_params_))
])

prepare_select_and_predict_pipeline.fit(housing, housing_labels)

some_data = housing.iloc[:4]
some_labels = housing_labels.iloc[:4]
print("Predictions:\t", prepare_select_and_predict_pipeline.predict(some_data))
print("Labels:\t\t", list(some_labels))

In [ ]:
# Explore preparation options in GridSearchCV

param_grid = [
        {'preparation__num_pipeline__imputer__strategy': ['mean', 'median', 'most_frequent'],
         'feature_selection__k': list(range(1, len(feature_importances) + 1)),
         'svm_reg__kernel': ['linear', 'rbf']}
]

grid_search_prep = GridSearchCV(prepare_select_and_predict_pipeline, param_grid, cv=3,
                                scoring='neg_mean_squared_error', verbose=2, n_jobs=4)
grid_search_prep.fit(housing, housing_labels)
grid_search_prep.best_params_